<a href="https://colab.research.google.com/github/vvamsi91/IML_AS7/blob/main/IML_AS7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install ipython-autotime
%load_ext autotime


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.72 s (started: 2023-12-12 23:33:26 +00:00)


In [35]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns

time: 625 µs (started: 2023-12-12 23:33:31 +00:00)


In [36]:
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)
np.random.seed(123)

time: 559 µs (started: 2023-12-12 23:33:31 +00:00)


In [37]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

time: 2.85 ms (started: 2023-12-12 23:33:31 +00:00)


In [38]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Stack images and calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=0)


Files already downloaded and verified
time: 19.5 s (started: 2023-12-12 23:33:31 +00:00)


In [39]:


# CIFAR-10 dataset with normalization
cifar10 = datasets.CIFAR10(
    './data', train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2470, 0.2435, 0.2616])
    ]))


time: 1.11 s (started: 2023-12-12 23:33:51 +00:00)


In [40]:
# CIFAR-10 validation dataset with normalization
cifar10_val = datasets.CIFAR10(
     './data', train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                             std=[0.2470, 0.2435, 0.2616])
    ]))

time: 441 ms (started: 2023-12-12 23:33:52 +00:00)


In [41]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)

time: 801 µs (started: 2023-12-12 23:33:52 +00:00)


In [42]:

class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

time: 386 µs (started: 2023-12-12 23:33:52 +00:00)


In [43]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

model = Net().to(device)



time: 5.73 ms (started: 2023-12-12 23:33:52 +00:00)


In [44]:
numel_list = [param.numel() for param in model.parameters()]
total_parameters = sum(numel_list)

print("Total number of parameters:", total_parameters)
print("Number of parameters per layer:", numel_list)

Total number of parameters: 18354
Number of parameters per layer: [432, 16, 1152, 8, 16384, 32, 320, 10]
time: 4.13 ms (started: 2023-12-12 23:33:52 +00:00)


In [45]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        total_loss_train = 0.0

        for batch_imgs, batch_labels in train_loader:
            batch_imgs = batch_imgs.to(device=device)
            batch_labels = batch_labels.to(device=device)

            optimizer.zero_grad()
            batch_outputs = model(batch_imgs)
            batch_loss = loss_fn(batch_outputs, batch_labels)
            batch_loss.backward()
            optimizer.step()

            total_loss_train += batch_loss.item()

        if epoch == 1 or epoch % 2 == 0:
            average_loss_train = total_loss_train / len(train_loader)
            print('{} Epoch {}, Training loss {:.4f}'.format(
                datetime.datetime.now(), epoch, average_loss_train))


time: 1.12 ms (started: 2023-12-12 23:33:52 +00:00)


In [46]:

# Define the training DataLoader, model, optimizer, and loss function
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
model = Net().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

# Train the model using the training loop
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
)

2023-12-12 23:34:17.773583 Epoch 1, Training loss 2.0681
2023-12-12 23:34:32.041590 Epoch 2, Training loss 1.7976
2023-12-12 23:35:02.212258 Epoch 4, Training loss 1.5253
2023-12-12 23:35:31.123083 Epoch 6, Training loss 1.3815
2023-12-12 23:35:59.927294 Epoch 8, Training loss 1.2717
2023-12-12 23:36:28.583565 Epoch 10, Training loss 1.1939
2023-12-12 23:36:59.030233 Epoch 12, Training loss 1.1361
2023-12-12 23:37:28.383222 Epoch 14, Training loss 1.0900
2023-12-12 23:37:57.413758 Epoch 16, Training loss 1.0554
2023-12-12 23:38:27.512185 Epoch 18, Training loss 1.0226
2023-12-12 23:38:58.180427 Epoch 20, Training loss 0.9953
2023-12-12 23:39:27.323946 Epoch 22, Training loss 0.9691
2023-12-12 23:39:56.649290 Epoch 24, Training loss 0.9483
2023-12-12 23:40:26.333250 Epoch 26, Training loss 0.9291
2023-12-12 23:40:55.851253 Epoch 28, Training loss 0.9137
2023-12-12 23:41:26.506516 Epoch 30, Training loss 0.9005
2023-12-12 23:41:56.216791 Epoch 32, Training loss 0.8850
2023-12-12 23:42:26

In [47]:
import collections

time: 446 µs (started: 2023-12-13 00:43:23 +00:00)


In [48]:
# DataLoader configurations
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

# Dictionary to store accuracy results
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    acc_dict = {}
    all_predictions = []
    all_expected_labels = []

    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)  # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

                all_predictions.extend(predicted.cpu().numpy())
                all_expected_labels.extend(labels.cpu().numpy())

        accuracy = correct / total
        print("Accuracy {}: {:.2f}".format(name, accuracy))
        acc_dict[name] = accuracy

    return acc_dict, all_predictions, all_expected_labels

# Call the validate function with the specified loaders
all_acc_dict, all_predictions, all_expected_labels = validate(model, train_loader, val_loader)

Accuracy train: 0.78
Accuracy val: 0.61
time: 14.8 s (started: 2023-12-13 00:43:23 +00:00)


In [ ]:
accuracy, predictions, expected_labels = validate(model, train_loader, val_loader)

In [ ]:
accuracy

In [51]:
cnf_matrix = confusion_matrix(all_predictions, all_expected_labels)
cnf_matrix

array([[4597,   75,  214,   73,   92,   22,   13,   50,  199,  118],
       [ 104, 5328,   21,   32,   14,   13,   12,   23,  129,  382],
       [ 330,   36, 3918,  279,  409,  266,  107,  259,   79,   33],
       [ 114,   29,  225, 3435,  266,  861,   94,  297,   53,   75],
       [  85,   21,  396,  271, 3985,  232,   72,  515,   36,   35],
       [  46,   19,  180,  601,  116, 3896,   35,  342,   20,   34],
       [ 115,   95,  876, 1141,  885,  540, 5615,  224,  103,  122],
       [  50,   13,   54,   65,  137,  114,   12, 4187,   12,   47],
       [ 439,  136,   96,   67,   69,   33,   28,   37, 5254,  189],
       [ 120,  248,   20,   36,   27,   23,   12,   66,  115, 4965]])

time: 40.1 ms (started: 2023-12-13 00:43:53 +00:00)


In [ ]:
print(classification_report(predictions, expected_labels, target_names=class_names))

In [53]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 64, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 4, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4 * 4 * 4, 32)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = out.view(-1, 4 * 4 * 4)
        out = self.act4(self.fc1(out))
        out = self.fc2(out)
        return out

time: 1.15 ms (started: 2023-12-13 00:43:54 +00:00)


In [54]:
model2 = Net2().to(device)

time: 4.47 ms (started: 2023-12-13 00:43:54 +00:00)


In [55]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
model2, optimizer2, loss_fn = Net2().to(device), optim.SGD(Net2().parameters(), lr=1e-2), nn.CrossEntropyLoss()
training_loop(300, optimizer2, model2, loss_fn, train_loader)

2023-12-13 00:44:08.530193 Epoch 1, Training loss 2.3086
2023-12-13 00:44:23.018266 Epoch 2, Training loss 2.3087
2023-12-13 00:44:52.291589 Epoch 4, Training loss 2.3086
2023-12-13 00:45:22.147918 Epoch 6, Training loss 2.3087
2023-12-13 00:45:50.602634 Epoch 8, Training loss 2.3087
2023-12-13 00:46:20.212820 Epoch 10, Training loss 2.3087
2023-12-13 00:46:49.088391 Epoch 12, Training loss 2.3087
2023-12-13 00:47:18.139097 Epoch 14, Training loss 2.3086
2023-12-13 00:47:47.819944 Epoch 16, Training loss 2.3087
2023-12-13 00:48:17.065250 Epoch 18, Training loss 2.3087
2023-12-13 00:48:46.451224 Epoch 20, Training loss 2.3087
2023-12-13 00:49:15.429004 Epoch 22, Training loss 2.3086
2023-12-13 00:49:44.528088 Epoch 24, Training loss 2.3086
2023-12-13 00:50:14.434390 Epoch 26, Training loss 2.3087
2023-12-13 00:50:43.422190 Epoch 28, Training loss 2.3087
2023-12-13 00:51:13.216901 Epoch 30, Training loss 2.3087
2023-12-13 00:51:41.928337 Epoch 32, Training loss 2.3087
2023-12-13 00:52:11

In [ ]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)
accuracy2, predictions2, expected_labels2 = validate(model2, train_loader, val_loader)
accuracy2

In [ ]:
cnf_matrix2 = confusion_matrix(predictions2, expected_labels2)
cnf_matrix2

In [59]:
print(classification_report(predictions2, expected_labels2, target_names=class_names))

              precision    recall  f1-score   support

    airplane       0.00      0.00      0.00         0
  automobile       0.00      0.00      0.00         0
        bird       1.00      0.10      0.18     59972
         cat       0.00      0.00      0.00         0
        deer       0.00      0.00      0.00         0
         dog       0.00      0.25      0.00        28
        frog       0.00      0.00      0.00         0
       horse       0.00      0.00      0.00         0
        ship       0.00      0.00      0.00         0
       truck       0.00      0.00      0.00         0

    accuracy                           0.10     60000
   macro avg       0.10      0.04      0.02     60000
weighted avg       1.00      0.10      0.18     60000

time: 171 ms (started: 2023-12-13 02:00:25 +00:00)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
